# Toxic Comment Classification

The goal of this project was to train a model that will classify comments as either toxic or non-toxic for moderation purposes.

The dataset includes the text of comments as well as their labels (i.e. toxic or not). The metric used to evaluate the final model was F1 score.

This task could have been approached in a variety of ways, but I decided to train a neural network using BERT to achieve the final goal. As I am new to working with neural networks, I attempted to explain the purpose of different steps in an effort to understand them better myself. Furthermore, I was unsuccessful in my first attempt to train a network, but I decided to leave the code from the first attempt at the bottom of this project.

*Note: Like with other projects, much of the code is commented to avoid rerunning code unnecessarily during later edits.*

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Импорты" data-toc-modified-id="Импорты-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Импорты</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовка-BERTа" data-toc-modified-id="Подготовка-BERTа-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Подготовка BERTа</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Проверка-sklearn-моделей" data-toc-modified-id="Проверка-sklearn-моделей-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Проверка sklearn моделей</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#XGBClassifier" data-toc-modified-id="XGBClassifier-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>XGBClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Лучшие-модели" data-toc-modified-id="Лучшие-модели-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Лучшие модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Первая-попытка" data-toc-modified-id="Первая-попытка"><span class="toc-item-num">3&nbsp;&nbsp;</span>Первая попытка</a></span></li></ul></div>

### Импорты

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import optuna


from tqdm import tqdm

import torch
from pytorch_transformers import BertConfig, DistilBertConfig
import transformers
from transformers import BertModel, DistilBertModel
from transformers import BertTokenizer, DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

# Classifiers

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


import warnings
warnings.filterwarnings('ignore')

import pdb

### Загрузка данных

In [2]:
try:
    comments = pd.read_csv("datasets/toxic_comments.csv")
    
except:
    comments = pd.read_csv("/datasets/toxic_comments.csv")

In [3]:
comments.info()
display(comments.head())
print(f'Количество токсичных комментариев: {len(comments.query("toxic == 1"))}')
print(f'Количество не токсичных комментакиев: {len(comments.query("toxic == 0"))}')
print(f'Процент токсичных комментариев в выборка: {len(comments.query("toxic == 1"))/len(comments.query("toxic == 0"))*100}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Количество токсичных комментариев: 16225
Количество не токсичных комментакиев: 143346
Процент токсичных комментариев в выборка: 11.31876717871444


In [4]:
print([comm for comm in comments if r'[а-яА-Я]+' in comm])

[]


**Загрузка и посмотр данных**

Данные в хорошем состоянии, а их много (159571). Столбца только два – один с комментариями, а другой с оценкой токсичности. Пропусков нет. Все комментарии на английском.

Однако, наблюдается значительный дисбаланс классов (примерно 1:9). Это может влиять на обучение и итоговое качество модели, поэтому нужно избавиться от такого дисбаланса. Это часто делается с помощью upsampling или downsampling, но из-за изначального размера этого датасета, логично делать downsampling (что в итоге уменьшит общее количество данных).

### Подготовка BERTа

Я решил использовать BERT для этого проекта. Я использовал bert-base-uncased; это (сравнительно) маленькая версия BERTа, а она не учитывает регистра слов. Я не стал изменить ничего в модели, то есть использовал и предобученное BERT, и использовал стандартную конфигурацию.


Из-за того, что я использовал BertTokenizer, не пришлось отдельно заниматься паддингом или маской.

### Подготовка данных

Здесь я создал класс для данных. Это сделано для удобства; это способствовало эмбеддингу ниже, но и также было легче экспериментировать с данными и в них разобраться (по второй этой причине есть в классе столько ненужных для задачи методов).

In [9]:
train, test = train_test_split(comments, test_size=0.25)


In [10]:
def downsample(df):
    
    fraction = len(df.query('toxic == 1'))/len(df.query('toxic == 0'))
    
    toxic = df[df['toxic'] == 1]
    non_toxic = df[df['toxic'] == 0]
    
    downsampled = pd.concat([toxic] + [non_toxic.sample(frac=fraction, random_state=12345)])
    
    downsampled = shuffle(downsampled, random_state=12345)

    return downsampled
    
train_downsampled = downsample(train)

print(f'Количество токсичных комментариев после downsampling: {len(train_downsampled.query("toxic == 1"))}')
print(f'Количество не токсичных комментариев после downsampling: {len(train_downsampled.query("toxic == 0"))}')

index_check = list(train_downsampled.head(10).index)
assert train['toxic'][index_check].values.all() == train_downsampled['toxic'][index_check].values.all()

train = train_downsampled

Количество токсичных комментариев после downsampling: 12064
Количество не токсичных комментариев после downsampling: 12064


In [11]:
print(f'Количество комментариев в train после downsampling: {len(train)}')
print(f'Количество комментариев в test: {len(test)}')

Количество комментариев в train после downsampling: 24128
Количество комментариев в test: 39893


In [12]:
class SamplingDataset(Dataset):

    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_token_len: int):
        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        
        ### max_token_len возможно, что это важно для определённых задач, но для этого проекта я использовал 512, то есть
        ### максимум, что bert может

        
    def __len__(self):
        return len(self.data)
    

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        sent = data_row['text']
        label = data_row['toxic'].flatten()

        encoding = self.tokenizer.encode_plus(

            sent,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',

        )

        
        ### это не только возвращает данные но и применяет tokenizer 
        
        
        return dict(
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            labels=torch.tensor(label, dtype=torch.long))

In [13]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


train_dataset = SamplingDataset(train, bert_tokenizer, 64)
val_dataset = SamplingDataset(test, bert_tokenizer, 64)


bert_train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True,
                              num_workers=0)

bert_eval_dataloader = DataLoader(val_dataset, batch_size=16, num_workers=0)


    ### DataLoader здесь принимает:

    # данные в форме Dataset (поэтому использовано torch.utils.data.Dataset как база при выявлении класса SamplingDataset)

    # batch_size > размер батча. Я читал, что чаще всего рекомендуют батчсайз 32. Тоже читал, что при меньших батчах, 
    # быстрее обрабатывается данные, но бывает, что тогда точность может пострадать.

    # num_workers > если num_workers=0, тогда процесс передачы данных (data fetching) выпускается в том же процессе как
    # и сам DataLoader. Из-за этого этот процесс может остановить общий процесс вычисления (вроде так, было 
    # написано "may block computing").
    # Когда num_workers какое-нибудь положительное число, тогда создано столько процессов для параллелной обработки.
    


In [14]:
class ClassBert(nn.Module):
    def __init__(self, model_path=None, config=None, num_labels=2):
        super(ClassBert, self).__init__() # super() даёт возможность использовать class (и их параметры) при создании других class-ов
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained(model_path)
        self.dropout = nn.Dropout(0.5) # dropout – переписывает часть тенсора нулями; здесь 0.5 > половина чисел в тенсоре будут переписаны как 0
        self.linear1 = nn.Linear(768, 256)
        self.linear2 = nn.Linear(256, self.num_labels)
                                # 768 – hidden size самой модели bert-base, 2 – количество классов (toxic или non-toxic)
                                # это подготавливает матрицы для того, чтобы работать с ними дальше (для forward propagation)
                                # не понимаю, откуда берётся 256, но заметно, что 256 становится первым изменернием в Linear2
        self.relu = nn.ReLU() # ReLu это функция для активации, которая делает сеть не-линейной.
                                # Как я понимаю, слои линейные, но не хотим, чтобы сеть из них тоже получилась линейной, ReLU помогает с этим

    def forward(self, input_ids, input_mask):

        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=input_mask, return_dict=False) #1

        dropout_output = self.dropout(pooled_output) #2
        linear_output = self.linear1(dropout_output) #3
        final_layer = self.relu(linear_output) #4
        final_layer = self.linear2(final_layer) #3 ещё раз?
        return final_layer
    
    
        ''' 
        1: преобразование данных в форму bert (transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions)
            - это работает с первым созданным классом Dataset
                - от класса Dataset получаем input_id и attention_mask
            - возвращает два torch.Tensor
            
        2: берёт эти тенсоры и применяет dropout, переписывая часть (здесь: половина) чисел в тенсоре в 0
        
        3: преобразует данные используя y = x(A^t) + b
            - важно, что меняет размер последного измерение
                - здесь, получится, что последное измерение (X в torch.size([a, X]) будет 2, поскольку это соответствует нашей задаче
                
            - преобразование final_layer
                - это интересно, а этого не было в моей первой попытке для этого проекта
                - видно, что у linear1 второе измерение 256, а у linear2 – 2
                    - 2 (linear2) конечно соответствует нашей задаче (у нас две группы комментариев – токсичные и не токсичные)
                    - 256 это - видимо это нужно, чтобы подготавливать данные к применению .relu
                        
        4: способствует работе сети для сложных задач, предотвращая линейной зависимости между слоями сети
         '''


In [21]:
bert_model = ClassBert(model_path='bert-base-uncased')

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu") 

learning_rate = 1e-05
bert_optimizer = torch.optim.Adam(params=bert_model.parameters(), lr=learning_rate)
# Adam это алгоритм для оптимтзации, тут lr влияет на то, насколько быстро/значительно меняется градиент ()
criterion = nn.CrossEntropyLoss()
bert_scheduler = torch.optim.lr_scheduler.StepLR(bert_optimizer, step_size=2, gamma=0.1)
# это работает с bert_optimizer, меняя lr постепенно, с каждой эпохой
n_epochs=4




def train_one_epoch(model, train_dataloader, criterion, optimizer, device=device):

    model.to(device).train()
    with tqdm(total=len(train_dataloader)) as pbar: #pbar это progress bar; рантьше я использовал tqdm с циклом, 
                                                   #  а pbar позволяет tqdm работать с len(train_dataloader) вместо цикла
        for batch in train_dataloader:
            # добавляем батч для вычисления на GPU
            # Распаковываем данные из dataloader
            input_ids, attention_mask, labels = batch
            # класс Dataset (здесь через DataLoader) как раз возвращает: input_ids, attention_mask, labels
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].view(-1).to(device)

            optimizer.zero_grad() # это важно чтобы восстановмит значение градиента (чтобы значения от предыдущего батча не остались)
            output = model.forward(input_ids, attention_mask)
            
            _, predicted = torch.max(output, 1)
            
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            # как я понимаю, loss.backward() и optimizer.step() чаще всего (всегда?) применяюься вместе, как раз в конце каждого
            # такого цикла. Они связаны с работой criterion и optimizer, то есть именно с тем, как они работают внутри себя.
            # .backward() вычисляет градиент, используя все предыдущие значения
            # .step() обновляет значение, используемое в градиенте
            
            _, predicted = torch.max(output.detach(), 1)
            f1 = f1_score(predicted.cpu().numpy(), labels.cpu().numpy(), zero_division=0)
            pbar.set_description('Loss: {:.4f}; F1_score: {:.4f}'.format(loss.item(), f1))    
            pbar.update(1)

def predict(model, val_dataloader, criterion, device=device):
    # в целом predict почти то же самое, как train_one_epoch
    # главное отличие в том, что тут нет оптимизации/нет градиента
    model.to(device).eval()
    losses = []
    predicted_classes = []
    true_classes = []
    # эти будут содержать информацию (про losses (не уверен, какой тип возвращает criterion),
    # предсказания о классе (токсикичный или нет), и настоящий класс (токсичный или нет)),
    # а в итоге predict возвращает эти три списка
    with tqdm(total=len(val_dataloader)) as pbar:
        with torch.no_grad():
            for batch in val_dataloader:
              
                input_ids, attention_mask, labels = batch
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].view(-1).to(device)


                output = model.forward(input_ids, attention_mask)
                _, predicted = torch.max(output, 1)

                loss = criterion(output, labels)
                losses.append(loss.item())
                _, predicted = torch.max(output.detach(), 1)
                predicted_classes.append(predicted)
                true_classes.append(labels)


                f1 = f1_score(predicted.cpu().numpy(), labels.cpu().numpy(), zero_division=0)
                pbar.set_description('Loss: {:.4f}; F1_score: {:.4f}'.format(loss.item(), f1))    
                pbar.update(1)
                
    predicted_classes = torch.cat(predicted_classes).detach().to('cpu').numpy()
    true_classes = torch.cat(true_classes).detach().to('cpu').numpy()
    return losses, predicted_classes, true_classes

def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=5, scheduler=None):
    model.to(device)
    for epoch in range(n_epochs):
        print('Learning rate: ', optimizer.param_groups[0]['lr'])
        print('Epoch:', epoch)
        train_one_epoch(model, train_dataloader, criterion, optimizer)
        print('Validation')
        losses, predicted_classes, true_classes = predict(model, val_dataloader, criterion)
        print('F1_score: ', f1_score(true_classes, predicted_classes, zero_division=0))
        scheduler.step()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
# pdb.set_trace()
train(bert_model, bert_train_dataloader, bert_eval_dataloader, criterion, bert_optimizer, device, n_epochs, bert_scheduler)

Learning rate:  1e-05
Epoch: 0


Loss: 0.0780; F1_score: 1.0000: 100%|█████| 1508/1508 [3:51:23<00:00,  9.21s/it]


Validation


Loss: 0.0528; F1_score: 0.0000: 100%|█████| 2494/2494 [1:46:24<00:00,  2.56s/it]


F1_score:  0.7542306178669815
Learning rate:  1e-05
Epoch: 1


...

F1_score:  0.73649226094388
Learning rate:  1.0000000000000002e-06
Epoch: 3


Loss: 0.1120; F1_score: 0.9412: 100%|█████| 1508/1508 [3:39:21<00:00,  8.73s/it]


Validation


Loss: 0.0038; F1_score: 0.0000: 100%|█████| 2494/2494 [1:47:10<00:00,  2.58s/it]

F1_score:  0.7287563308947664


Когда я первый раз пробовал делать эмбеддингы, ядро умерло после где-то 12 часов обработки. Чтобы избегать повторений такого случая, я разбил данные на 4 части (в конце концов решил соединить части 3 и 4).

Из-за этого процесса и размера батчов, я потерял 50 строк данных (то есть, 8112 строк части 1 стал эмбеддингом 8100 строк, и так далее). Из-за огромного количество времени, которое ушло на создание эмбеддингов, я не стал исправить это.

In [ ]:
fourth = len(comments)/4

print(fourth, fourth*2, fourth*3, fourth*4)

In [ ]:
data_1 = tensor_data(comments[0:8112])
data_2 = tensor_data(comments[8112:16225])
data_3_4 = tensor_data(comments[16225:])
# data_4 = tensor_data(comments[24338:])

print(len(data_1)+len(data_2)+len(data_3_4))

In [ ]:
do_embed = False

In [ ]:
if do_embed:
    batch_size = 100
    embeddings_1 = []

#     for i in tqdm(range(len(data_1)//batch_size)):
#         batch = data_1[batch_size*i:batch_size*(i+1)][0]
#         mask = data_1[batch_size*i:batch_size*(i+1)][1]
#         with torch.no_grad():
#             batch_embeddings = model(batch, attention_mask=mask)
#         embeddings_1.append(batch_embeddings[0][:,0,:].numpy())
        
#     np.savez("datasets/batch_embeddings_1", *embeddings_1)
    

else:
    file_load_1 = np.load("datasets/batch_embeddings_1_done.npz", allow_pickle=True)
    embeddings_1 = [file_load_1[n] for n in file_load_1]


In [ ]:
if do_embed:
    batch_size = 100
    embeddings_2 = []

#     for i in tqdm(range(len(data_2)//batch_size)):
#         batch = data_2[batch_size*i:batch_size*(i+1)][0]
#         mask = data_2[batch_size*i:batch_size*(i+1)][1]
#         with torch.no_grad():
#             batch_embeddings = model(batch, attention_mask=mask)
#         embeddings_2.append(batch_embeddings[0][:,0,:].numpy())
        
#     np.savez("datasets/batch_embeddings_2", *embeddings_2)
    

else:
    file_load_2 = np.load("datasets/batch_embeddings_2_done.npz", allow_pickle=True)
    embeddings_2 = [file_load_2[n] for n in file_load_2]

In [ ]:
if do_embed:
    batch_size = 100
    embeddings_3_4 = []

#     for i in tqdm(range(len(data_3_4)//batch_size)):
#         batch = data_3_4[batch_size*i:batch_size*(i+1)][0]
#         mask = data_3_4[batch_size*i:batch_size*(i+1)][1]
#         with torch.no_grad():
#             batch_embeddings = model(batch, attention_mask=mask)
#         embeddings_3_4.append(batch_embeddings[0][:,0,:].numpy())
        
#     np.savez("datasets/batch_embeddings_3_4", *embeddings_3_4)
    

else:
    file_load_3_4 = np.load("datasets/batch_embeddings_3_4_done.npz", allow_pickle=True)
    embeddings_3_4 = [file_load_3_4[n] for n in file_load_3_4]

In [ ]:
features_1 = np.concatenate(embeddings_1)
target_1 = comments['toxic'][0:8100]

features_2 = np.concatenate(embeddings_2)
target_2 = comments['toxic'][8112:16212]

features_3_4 = np.concatenate(embeddings_3_4)
target_3_4 = comments['toxic'][16225:32425]

assert (len(features_1) == len(target_1)) & (len(features_2) == len(target_2)) & (len(features_3_4) == len(target_3_4))

features = np.concatenate((features_1, features_2, features_3_4))
target = pd.concat((target_1, target_2, target_3_4))

assert len(features) == len(target)



In [ ]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2)

train_target = np.array(train_target)
test_target = np.array(test_target)

assert (len(train_features) == len(train_target)) & (len(test_features) == len(test_target))

In [ ]:
print(f'Размер тренирочной выборки: {len(train_target)}')
print(f'Размер тестовой выборки: {len(test_target)}')

## Обучение

### Проверка sklearn моделей

- LogisticRegression
- DecisionTreeClassifier
- RandomForestClassifier

In [ ]:
log_reg = LogisticRegression(random_state=12345)
dec_tree = DecisionTreeClassifier(random_state=12345)
rand_for = RandomForestClassifier(random_state=12345)

model_list = [log_reg, dec_tree, rand_for]
model_names = ['logistic_regression', 'decision_tree', 'random_forest']

def sklearn_test(train_features, train_target, test_features, test_target):
    
    f1_results = {}
    
    for model, name in zip(model_list, model_names):
        model.fit(train_features, train_target)
        pred = model.predict(test_features)
        f1_results[f'{name}'] = list(cross_val_score(model, test_features, test_target, scoring='f1'))
        
    f1_df = pd.DataFrame.from_dict(f1_results, orient='index')
    f1_df.columns = ['cross_val_1', 'cross_val_2', 'cross_val_3', 'cross_val_4', 'cross_val_5']
    f1_df['f1_average'] = f1_df.apply(lambda x: sum(x)/len(x), axis=1)
    f1_df = f1_df.sort_values('f1_average', ascending=False)
    display(f1_df)
    

sklearn_test(train_features, train_target, test_features, test_target)

In [ ]:
log_reg = LogisticRegression(random_state=12345)
dec_tree = DecisionTreeClassifier(random_state=12345)
rand_for = RandomForestClassifier(random_state=12345)
dummy = DummyClassifier()

model_list = [log_reg, dec_tree, rand_for, dummy]
model_names = ['logistic_regression', 'decision_tree', 'random_forest', 'dummy']

def sklearn_test(train_features, train_target, test_features, test_target):
    
    f1_results = {}
        
    for model, name in zip(model_list, model_names):
        model.fit(train_features, train_target)
        pred = model.predict(test_features)
        f1 = f1_score(test_target, pred)
        f1_results[f'{name}'] = list(cross_val_score(model, train_features, train_target, scoring='f1'))
        f1_results[f'{name}'].append(f1)
        
        
    f1_df = pd.DataFrame.from_dict(f1_results, orient='index')
    f1_df.columns = ['cross_val_1', 'cross_val_2', 'cross_val_3', 'cross_val_4', 'cross_val_5', 'test_f1']
    f1_df = f1_df.sort_values('test_f1', ascending=False)
    display(f1_df)
    

sklearn_test(train_features, train_target, test_features, test_target)

In [ ]:
lgb_class = LGBMClassifier()
xgb_class = XGBClassifier()
cat_class = CatBoostClassifier()
model_list = [lgb_class, xgb_class, cat_class]
model_names = ['lgb_class', 'xgb_class', 'cat_class']

def prelim_test(train_features, train_target, test_features, test_target):
    
    f1_results = {}
    
    for model, name in zip(model_list, model_names):
        model.fit(train_features, train_target, verbose=False)
        pred = model.predict(test_features)
        f1 = f1_score(test_target, pred)
        f1_results[f'{name}'] = f1
        
    f1_df = pd.DataFrame.from_dict(f1_results, orient='index')
    f1_df.columns = ['f1_score']
    f1_df = f1_df.sort_values('f1_score', ascending=False)
    display(f1_df)

prelim_test(train_features, train_target, test_features, test_target)

### LGBMClassifier

In [ ]:
def objective_lgb(trial):
    
    params = {
        'boosting_type' : 'gbdt',
        'n_estimators' : trial.suggest_int('n_estimators', 50, 500, 25),
        'learning_rate' : trial.suggest_float('learning_rate', 0.05, 0.2),
        'max_depth' : trial.suggest_int('max_depth', 1, 20, 2),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 50, 1000, 50),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 5),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 300, 10),
        'verbose' : -1
    }
    lgb_mod = LGBMClassifier(**params, random_state=12345)
    lgb_mod.fit(train_features, train_target)
    pred = lgb_mod.predict(test_features)
    f1 = f1_score(test_target, pred)
    return f1

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=20)


print("Number of finished trials: {}".format(len(study_lgb.trials)))

print("Best trial:")
trial_lgb = study_lgb.best_trial

print("  Value: {}".format(trial_lgb.value))

print("  Params: ")
for key, value in trial_lgb.params.items():
    print("    {}: {}".format(key, value))


display(optuna.importance.get_param_importances(study_lgb))

Number of finished trials: 20

Best trial:

  Value: 0.8901662265030293
  
  Params: 
  
    n_estimators: 500
    learning_rate: 0.15060263548375816
    max_depth: 17
    reg_alpha: 6.724134652658169e-06
    reg_lambda: 7.488943400676552
    num_leaves: 200
    colsample_bytree: 0.9352407649941042
    subsample: 0.8502365864897415
    subsample_freq: 2
    min_child_samples: 180

### XGBClassifier

In [ ]:
 def objective_xgb(trial):
        
        params = {
         'n_estimators' : trial.suggest_int('n_estimators', 5, 150),
         'max_depth' : trial.suggest_int('max_depth', 2, 16),
         'max_leaves' : trial.suggest_int('max_leaves', 30, 100),
         'learning_rate' : trial.suggest_float('learning_rate', 0.1, 1),
         'reg_alpha' : trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
         'reg_lambda' : trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
         'verbosity' : 0,
         'booster' : 'gbtree'
        }
        xgb_class = XGBClassifier(**params, random_state=12345)
        xgb_class.fit(train_features, train_target)
        pred = xgb_class.predict(test_features)
        f1 = f1_score(test_target, pred)
        return f1
    
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=20)

print("Number of finished trials: {}".format(len(study_xgb.trials)))

print("Best trial:")
trial_xgb = study_xgb.best_trial

print("  Value: {}".format(trial_xgb.value))

print("  Params: ")
for key, value in trial_xgb.params.items():
    print("    {}: {}".format(key, value))
    
display(optuna.importance.get_param_importances(study_xgb))


Number of finished trials: 20

Best trial:

  Value: 0.09294320137693632
  
  Params: 
  
    n_estimators: 5
    max_depth: 13
    max_leaves: 76
    learning_rate: 0.853680753351705
    reg_alpha: 0.02833383057451046
    reg_lambda: 0.022508569427159838

### CatBoostClassifier

CatBoostClassifier дал хороший изначальный результат, но он очень долго у меня обрабатывается. Я пробовал использовать optuna как в ячейке ниже пару раз, с малым количеством n_trials, но результат был хуже чем с стандартными параметрами. Я уверен, что CatBoostClassifier может дать результат по лучше, но из-за долгого времени обработки я не стал сыскать лучшие параметры чем стандартные.

In [ ]:
# def objective_cat(trial):
    
#     params = {
#         'iterations' : trial.suggest_int('iterations', 10, 100, 10),
#         'depth' : trial.suggest_int('depth', 1, 16),
#         'learning_rate' : trial.suggest_float('learning_rate', 0.1, 1),
#         'subsample' : trial.suggest_float('subsample', 0.6, 1.0),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'early_stopping_rounds' : 70,
#         'eval_metric' : 'F1',
#         'silent' : True
#     }
    
#     cat_mod = CatBoostClassifier(**params, random_state=12345)
#     cat_mod.fit(train_features, train_target)
#     pred = cat_mod.predict(test_features)
#     f1 = f1_score(test_target, pred)
#     return f1

# study_cat = optuna.create_study(direction='maximize')
# study_cat.optimize(objective_cat, n_trials=5)


# print("Number of finished trials: {}".format(len(study_cat.trials)))

# print("Best trial:")
# trial_cat = study_cat.best_trial

# print("  Value: {}".format(trial_cat.value))

# print("  Params: ")
# for key, value in trial_cat.params.items():
#     print("    {}: {}".format(key, value))


# display(optuna.importance.get_param_importances(study_cat))



### Лучшие модели

In [ ]:
best_f1 = {}

### CatBoost

best_cat = CatBoostClassifier(silent=True, random_state=12345)

best_cat.fit(train_features, train_target)
pred = best_cat.predict(test_features)

f1 = f1_score(test_target, pred)
best_f1['CatBoost'] = f1


### LGBMClassifier

best_params_lgb = {"boosting_type" : 'gbdt', 
                   "verbosity" : -1, 
                   "n_estimators" : 500, 
                   "learning_rate" : 0.15060263548375816, 
                   "max_depth" : 17,
                   "reg_alpha" : 6.724134652658169e-06,
                   "reg_lambda" : 7.488943400676552,
                   "num_leaves" : 200,
                   "colsample_bytree" : 0.9352407649941042,
                   "subsample" : 0.8502365864897415,
                   "subsample_freq" : 2,
                   "min_child_samples" : 180
}

best_lgb = LGBMClassifier(**best_params_lgb, random_state=12345)

best_lgb.fit(train_features, train_target)
pred = best_lgb.predict(test_features)

f1 = f1_score(test_target, pred,)
best_f1['LGBM'] = f1

### XGBoost

best_params_xgb = {"verbosity" : 0,
                   "booster" : 'gbtree',
                   "n_estimators" : 5,
                   "max_depth" : 13,
                   "max_leaves" : 76,
                   "learning_rate" : 0.853680753351705,
                   "reg_alpha" : 0.02833383057451046,
                   "reg_lambda" : 0.022508569427159838
}

best_xgb = XGBClassifier(**best_params_xgb, random_state=12345)

best_xgb.fit(train_features, train_target)
pred = best_xgb.predict(test_features)

f1 = f1_score(test_target, pred)
best_f1['XGBoost'] = f1


best_models_df = pd.DataFrame.from_dict(best_f1, orient="index")
best_models_df.columns = ['f1']
best_models_df = best_models_df.sort_values(by='f1')
display(best_models_df)



In [ ]:
best_models_df = best_models_df.sort_values(by='f1', ascending=False)
display(best_models_df)

## Выводы

BERT дал очень хороший результат:

Лучшая модель: LGBMClassifier

F1 score: 0.8875

Параметры:

    "boosting_type" : 'gbdt', 
    "verbosity" : -1, 
    "n_estimators" : 500, 
    "learning_rate" : 0.15060263548375816, 
    "max_depth" : 17,
    "reg_alpha" : 6.724134652658169e-06,
    "reg_lambda" : 7.488943400676552,
    "num_leaves" : 200,
    "colsample_bytree" : 0.9352407649941042,
    "subsample" : 0.8502365864897415,
    "subsample_freq" : 2,
    "min_child_samples" : 180
